### Packages

In [3]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import pmdarima
# import warnings
# warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [6]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2018-01-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, threads = True)

[*********************100%***********************]  4 of 4 completed


In [7]:
df = raw_data.copy()

In [8]:
df['spx'] = df['^GSPC'].Close[:]
df['dax'] = df['^GDAXI'].Close[:]
df['ftse'] = df['^FTSE'].Close[:]
df['nikkei'] = df['^N225'].Close[:]

In [9]:
df = df.iloc[1:]
del df['^N225']
del df['^GSPC']
del df['^GDAXI']
del df['^FTSE']
df=df.asfreq('b')
df=df.fillna(method='ffill')

### Creating Returns

In [10]:
df['ret_spx'] = df.spx.pct_change(1)*100
df['ret_ftse'] = df.ftse.pct_change(1)*100
df['ret_dax'] = df.dax.pct_change(1)*100
df['ret_nikkei'] = df.nikkei.pct_change(1)*100

### Splitting the Data

In [15]:
df = df.droplevel(1, axis=1)

In [16]:
size = int(len(df)*0.8)
df_train, df_test = df.iloc[:size], df.iloc[size:]

In [17]:
df_train.head()

,spx,dax,ftse,nikkei,ret_spx,ret_ftse,ret_dax,ret_nikkei
Date,,,,,,,,
1994-01-10,475.269989,2225.000000,3440.600098,18443.439453,NaN,NaN,NaN,NaN
1994-01-11,474.130005,2228.100098,3413.800049,18485.250000,-0.239860,-0.778935,0.139330,0.226696
1994-01-12,474.170013,2182.060059,3372.000000,18793.880859,0.008438,-1.224443,-2.066336,1.669606
1994-01-13,472.470001,2142.370117,3360.000000,18577.259766,-0.358524,-0.355872,-1.818921,-1.152615
1994-01-14,474.910004,2151.050049,3400.600098,18973.699219,0.516435,1.208336,0.405156,2.134004


### Fitting a Model

In [25]:
from pmdarima.arima import auto_arima

In [26]:
model = auto_arima(df_train['ret_ftse'][1:])

/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in r

In [27]:
model

ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [28]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -7882.776
Date:                Thu, 12 Jan 2023   AIC                          15785.552
Time:                        17:18:27   BIC                          15850.762
Sample:                    01-11-1994   HQIC                         15808.403
                         - 04-05-2013                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0121      0.082      0.148      0.882      -0.148       0.172
ar.L2         -0.6541      0.077     -8.455      0.000      -0.806      -0.502
ar.L3         -0.1627      0.071     -2.289      0.022      -0.302      -0.023
ar.L4          0.2016      0.074      2.714      0.007       0.056       0.347
ma.L1         -0.0358      0.081     -0.441      0.659      -0.195       0.123
ma.L2          0.6066      0.078      7.767      0.000       0.453       0.760
ma.L3          0.0621      0.068      0.907      0.364      -0.072       0.196
ma.L4         -0.1935      0.073     -2.652      0.008      -0.337      -0.051
ma.L5         -0.1052      0.010    -11.066      0.000      -0.124      -0.087
sigma2         1.3542      0.015     91.443      0.000       1.325       1.383
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              6354.74
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               1.99   Skew:                            -0.20
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

    - Auto ARIMA only considers a single feature - the AIC
    - We could have easily overfitted while going through the models in our previous sections
    - The default arguments of the method restrict the number of AR and MA components

### Important Arguments

In [8]:
# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# return_valid_fits -> whether or not the method should validate the results 
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple_size -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [31]:
model = auto_arima(df_train['ret_ftse'][1:], exogenous = df_train[['ret_spx', 'ret_dax', 'ret_nikkei']][1:],
                   m = 5, max_order=None, max_p=7, max_q=7, max_d=2, max_P=4, max_Q=4, max_D=2, maxiter=100, alpha=0.05, 
                   n_jobs=-1, trend='ct')

/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/pmdarima/arima/_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace

In [32]:
model = auto_arima(df['ret_ftse'][1:], exogenous = df[['ret_spx', 'ret_dax', 'ret_nikkei']][1:],
                   m = 5, max_order=None, max_p=7, max_q=7, max_d=2, max_P=4, max_Q=4, max_D=2, maxiter=100, alpha=0.05, 
                   n_jobs=-1, trend='ct', information_criterion='aicc', out_of_sample_size=int(len(df)*0.2))

/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/pmdarima/arima/_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/tim/miniforge3/envs/tf_m1/lib/python3.8/site-packages/statsmodels/tsa/statespace

In [33]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                 6274
Model:             SARIMAX(2, 0, 3)x(0, 0, [1], 5)   Log Likelihood               -9575.874
Date:                             Sat, 14 Jan 2023   AIC                          19169.748
Time:                                     04:21:53   BIC                          19230.445
Sample:                                          0   HQIC                         19190.779
                                            - 6274                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0264      0.045      0.587      0.557      -0.062       0.114
drift      -2.202e-06   1.28e-05     -0.173      0.863   -2.72e-05    2.28e-05
ar.L1         -0.3551      0.106     -3.356      0.001      -0.562      -0.148
ar.L2         -0.1441      0.119     -1.210      0.226      -0.377       0.089
ma.L1          0.3309      0.105      3.141      0.002       0.124       0.537
ma.L2          0.0852      0.121      0.707      0.480      -0.151       0.321
ma.L3         -0.1058      0.009    -11.929      0.000      -0.123      -0.088
ma.S.L5       -0.0516      0.015     -3.445      0.001      -0.081      -0.022
sigma2         1.3653      0.014     94.610      0.000       1.337       1.394
===================================================================================
Ljung-Box (L1) (Q):                   0.11   Jarque-Bera (JB):              8552.01
Prob(Q):                              0.74   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                            -0.19
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.71
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""